In [38]:
import numpy as np
import pandas as pd

from selenium import webdriver as wb
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time 
from tqdm import tqdm

import requests
from datetime import datetime
import json
import pickle

## 데이터 불러오기

In [39]:
df = pd.read_csv("data/request_reviews.csv")
df.head(3)

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,203520.881091,446950.285626,NaN,NaN,청담동,NaN,배초향,NaN,,
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,205130.591679,445590.096838,기타,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240


In [40]:
df.shape

(12528, 20)

In [41]:
df[df.keyword.isna()]

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,X,Y,TRDPJUBNSENM,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2
247,247,3220000-101-1990-07507,19901101,영업/정상,31.73,135860,서울특별시 강남구 도곡동 954-19번지,서울 강남구 남부순환로355길 13,(신)오양절면,한식,203087.887972,442628.722914,기타,상수도전용,도곡동,NaN,NaN,['338342608'],338342608,338342608
994,994,3220000-101-1998-12691,19981029,영업/정상,94.63,135515,서울특별시 강남구 역삼동 787-1번지,서울 강남구 논현로64길 22,(중화요리)남경,중국식,203668.426416,443485.604196,기타,상수도전용,역삼동,NaN,NaN,['10960059'],10960059,10960059
1432,1432,3220000-101-2000-22499,2000-09-27,영업/정상,677.61,135-731,서울특별시 강남구 삼성동 159 코엑스,서울 강남구 영동대로 513,(유)아웃백스테이크하우스코리아 코엑스스타필드점,경양식,205130.591679,445590.096838,NaN,NaN,삼성동,NaN,NaN,['16877892'],16877892,16877892
3866,3866,3220000-101-2011-00114,20110223,영업/정상,81.78,135955,서울특별시 강남구 청담동 122-7 1층,서울 강남구 도산대로85길 14,(첼시스 하이볼) Chelsea's HIGHBALL,일식,204279.144297,447023.007955,NaN,NaN,청담동,NaN,NaN,['19153490'],19153490,19153490
4031,4031,3220000-101-2011-00686,20110811,영업/정상,49.50,135515,서울특별시 강남구 역삼동 781-41 지상1층102호,서울 강남구 논현로66길 9,(SOS)커피 역삼점,경양식,203566.263883,443554.896146,NaN,NaN,역삼동,NaN,NaN,['14600986'],14600986,14600986
4511,4511,3220000-101-2013-00033,20130117,영업/정상,177.28,135911,서울특별시 강남구 역삼동 646-4번지,서울 강남구 강남대로94길 86,(이층) 맥주창고,경양식,202951.131852,444272.491202,NaN,NaN,역삼동,NaN,NaN,NaN,,
6054,6054,3220000-101-2016-00899,2016-10-21,영업/정상,507.39,135-825,서울특별시 강남구 논현동 165,서울 강남구 강남대로 502,(유)아웃백스테이크하우스코리아 강남점,경양식,202024.615630,444994.659485,NaN,NaN,논현동,NaN,NaN,['522500805'],522500805,522500805
8697,8697,3220000-101-2020-01279,20201030,영업/정상,268.98,135896,서울특별시 강남구 신사동 637-11,서울 강남구 압구정로42길 12,(끌레오)CLO,한식,203049.750409,447323.452649,NaN,NaN,신사동,NaN,NaN,['1495236287'],1495236287,1495236287
9148,9148,3220000-101-2021-00506,20210503,영업/정상,33.00,135949,서울특별시 강남구 청담동 9-12,서울 강남구 학동로77길 50,(유노시엘) EUNOSIERE,기타,204171.424581,446636.433530,NaN,NaN,청담동,NaN,NaN,['118575691'],118575691,118575691
9354,9354,3220000-101-2021-00858,20210708,영업/정상,159.25,135915,서울특별시 강남구 역삼동 676 센터필드,서울 강남구 테헤란로 231,(유)한국집 센터필드 역삼점,한식,203599.062772,444556.062267,NaN,NaN,역삼동,NaN,NaN,['1872560550'],1872560550,1872560550


In [42]:
# df["url"] = df["id_mod2"].apply(lambda x: np.nan if x==" " else x)
df["url"] = df["id_mod2"].fillna(" ")
df["url"] = df["url"].apply(lambda x: "https://place.map.kakao.com" + x if x!=" " else x)

In [43]:
df["keyword2"] = df["keyword"].fillna(" ")

# 업소 이름 뒤에 위치한 동 이름 붙임
def plus_dong(txt):
    if txt["dong"][:-1] not in txt["keyword2"]:
        txt["name"] = txt["keyword2"] + " " + txt["dong"][:-1]
    else:
        txt["name"] = txt["keyword2"]
    return txt

In [44]:
# plus_dong 함수 적용
df = df.apply(plus_dong, axis=1)

## request 적용 -> 실패

In [45]:
df.head(3)

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,...,NaN,청담동,NaN,배초향,NaN,,,,배초향,배초향 청담
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,...,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881,https://place.map.kakao.com10825881,브래서리,브래서리 삼성
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,...,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240,https://place.map.kakao.com727661240,로비라운지,로비라운지 삼성


In [46]:
df["id_mod2"][:3]

0             
1     10825881
2    727661240
Name: id_mod2, dtype: object

In [47]:
request_id = 14569639
url = f'https://place.map.kakao.com/main/v/{request_id}'
response = requests.get(url)
comments = []
if response.status_code == 200:
    comment_data = json.loads(response.content)['comment']
    comment_count = int(comment_data['kamapComntcnt'])
    overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
    for c in comment_data['list']:
        comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
        request_comment_id = c['commentid']
    print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
else:
    print('error')

총 댓글 개수: 169개, 평점: 4.11점


In [48]:
while len(comments) < comment_count:
    url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
    response = requests.get(url)
    if response.status_code == 200:
        comment_data = json.loads(response.content)['comment']
        for c in comment_data['list']:
            try:
                comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
            except:
                comments.append({'점수': c['point'], '댓글': '', '작성일': c['date']})
            request_comment_id = c['commentid']
    else:
        print('error')

In [49]:
pd.DataFrame(comments)

,점수,댓글,작성일
0,5,,2024.04.22.
1,2,너무 짜요. 처음엔 먹을만했는데. 절반정도 먹고나선 너무 짜서 짜증날 정도임. 맛 ...,2024.04.14.
2,5,곰탕 정말 빨리나옵니다. 주문하고 5분도 안되서 음식 받은거 같아요. 정말 맛있고 ...,2024.04.11.
3,5,수육이랑 곰탕먹었어요\n곰탕은 주문하자마자 빨리 나왔어요\n고기도 많이 들어가있고\...,2024.04.08.
4,5,세종대왕의 이름 (이도 李裪)에 걸맞는 \n진한 맛의 곰탕입니다\n마침 제옆에 외국...,2024.04.02.
...,...,...,...
164,4,곰탕,2016.09.29.
165,4,소수육46. 면+죽 추가10.,2016.09.09.
166,2,공장형.\n밥대신 국수.\n잘 모르겠다.,2016.08.03.
167,4,,2016.07.10.


In [50]:
comment_data

{'kamapComntcnt': 169,
 'scoresum': 695,
 'scorecnt': 169,
 'list': [{'commentid': '842665',
   'contents': '(수육+장칼국수+죽) 3명에서 이렇게 먹으니 딱좋네.\n복잡한 역삼역에 주차공간도 나름 넉넉 (10대정도?)\n다음에는 곰탕을 먹어 봐야겠다.',
   'point': 5,
   'username': '자유로',
   'profile': 'http://t1.daumcdn.net/local/kakaomapPhoto/profile/381d738b852dead3074f44ea9effa8bf24a6f1bb?original',
   'profileStatus': 'S',
   'photoCnt': 1,
   'likeCnt': 0,
   'thumbnail': 'https://t1.daumcdn.net/local/review_placeapp/oWaiP3LwQf_QimK4eF87DMjZwTKIiln20_lkagebjc44A.jpeg',
   'kakaoMapUserId': '267318248',
   'photoList': [{'url': 'https://t1.daumcdn.net/local/review_placeapp/oWaiP3LwQf_QimK4eF87DMjZwTKIiln20_lkagebjc44A.jpeg',
     'near': False}],
   'ownerReply': {},
   'userCommentCount': 72,
   'userCommentAverageScore': 4.2,
   'myStorePick': False,
   'level': {'nowLevel': 28, 'badge': '02'},
   'date': '2016.03.15.',
   'isMy': False,
   'isBlock': False,
   'isEditable': False,
   'isMyLike': False}],
 'strengthCounts': [{'id': 5, 'nam

In [51]:
reviews_df = pd.DataFrame(columns=("name", "user", 'review', 'date', 'rate', 
                                   "avg_rate", "review_cnt", "is_model"))
reviews_df

,name,user,review,date,rate,avg_rate,review_cnt,is_model


In [55]:
comments = []

for i, id in enumerate(df["id_mod2"][:2]):
    try:
        request_id = id
        url = f'https://place.map.kakao.com/main/v/{request_id}'
        response = requests.get(url)

        if response.status_code == 200:
            comment_data = json.loads(response.content)['comment']
            comment_count = int(comment_data['kamapComntcnt'])
            overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
            for c in comment_data['list']:
                comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
                request_comment_id = c['commentid']
            print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
        else:
            print('error')

        while len(comments) < comment_count:
            url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
            response = requests.get(url)
            if response.status_code == 200:
                comment_data = json.loads(response.content)['comment']
                for c in comment_data['list']:
                    try:
                        comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                        'review': c['contents'], 'date': c['date'],
                                        'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                        "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                    except:
                        comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
                                        'review': "", 'date': c['date'],
                                        'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
                                        "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
                    request_comment_id = c['commentid']
                reviews_df = pd.concat([reviews_df, pd.DataFrame(comments)])
            else:
                # print('error in while')
                pass


    except:
        pass

error
총 댓글 개수: 38개, 평점: 4.55점


In [52]:
# comments = []

# for i, id in enumerate(df["id_mod2"][:2]):
#     try:
#         request_id = id
#         url = f'https://place.map.kakao.com/main/v/{request_id}'
#         response = requests.get(url)

#         if response.status_code == 200:
#             comment_data = json.loads(response.content)['comment']
#             comment_count = int(comment_data['kamapComntcnt'])
#             overall_score = int(comment_data['scoresum']) / int(comment_data['scorecnt'])
#             for c in comment_data['list']:
#                 comments.append({'점수': c['point'], '댓글': c['contents'], '작성일': c['date']})
#                 request_comment_id = c['commentid']
#             print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
#         else:
#             print('error')
#         #     for c in comment_data['list']:
#         #         try:
#         #             comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#         #                             'review': c['contents'], 'date': c['date'],
#         #                             'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#         #                             "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#         #         except:
#         #             comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#         #                             'review': "", 'date': c['date'],
#         #                             'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#         #                             "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#         #         request_comment_id = c['commentid']
#         #     # print(f'총 댓글 개수: {comment_count}개, 평점: {np.round(overall_score, 2)}점')
#         #     # reviews_df = pd.concat([reviews_df, pd.DataFrame(comments)])
#         # else:
#         #     # print('error')
#         #     pass



#         while len(comments) < comment_count:
#             url = f'https://place.map.kakao.com/commentlist/v/{request_id}/{request_comment_id}'
#             response = requests.get(url)
#             if response.status_code == 200:
#                 comment_data = json.loads(response.content)['comment']
#                 for c in comment_data['list']:
#                     try:
#                         comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#                                         'review': c['contents'], 'date': c['date'],
#                                         'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#                                         "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#                     except:
#                         comments.append({"name": df.loc[i, "keyword"], "user": c["username"], 
#                                         'review': "", 'date': c['date'],
#                                         'rate': c['point'], "avg_rate": c["userCommentAverageScore"],
#                                         "review_cnt": c["userCommentCount"], "is_model": df.loc[i, "exemplary_diner"]})
#                     request_comment_id = c['commentid']
#                 # reviews_df = pd.concat([reviews_df, pd.DataFrame(comments)])
#             else:
#                 # print('error in while')
#                 pass


#     except:
#         pass

## selenium

In [57]:
df.head()

,Unnamed: 0,MGTNO,APVPERMYMD,TRDSTATENM,SITEAREA,SITEPOSTNO,SITEWHLADDR,RDNWHLADDR,BPLCNM,UPTAENM,...,WTRSPLYFACILSENM,dong,exemplary_diner,keyword,id,id_mod,id_mod2,url,keyword2,name
0,0,3220000-101-1007-00001,19970205,영업/정상,NaN,135954,서울특별시 강남구 청담동 88-5번지,서울 강남구 도산대로55길 20,배초향,한식,...,NaN,청담동,NaN,배초향,NaN,,,,배초향,배초향 청담
1,1,3220000-101-1899-20451,19991005,영업/정상,950.66,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,브래서리,경양식,...,상수도전용,삼성동,모범식당지정,브래서리,['10825881'],10825881,10825881,https://place.map.kakao.com10825881,브래서리,브래서리 삼성
2,2,3220000-101-1899-20453,19991005,영업/정상,473.24,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비라운지,경양식,...,상수도전용,삼성동,NaN,로비라운지,['727661240'],727661240,727661240,https://place.map.kakao.com727661240,로비라운지,로비라운지 삼성
3,3,3220000-101-1899-20463,19991006,영업/정상,282.59,135090,서울특별시 강남구 삼성동 159,서울 강남구 봉은사로 524,로비바,경양식,...,상수도전용,삼성동,NaN,로비바,['21550576'],21550576,21550576,https://place.map.kakao.com21550576,로비바,로비바 삼성
4,4,3220000-101-1899-20500,19991012,영업/정상,"1,157.90",135873,서울특별시 강남구 삼성동 109-6,서울 강남구 봉은사로 637,노블발렌티,한식,...,상수도전용,삼성동,NaN,노블발렌티,"['17157328', '26597965']",17157328,17157328,https://place.map.kakao.com17157328,노블발렌티,노블발렌티 삼성


In [ ]:
# 상세보기에 조회된 모범 음식점 목록을 조회

# store_name = "솥내음 스타필드 코엑스몰"
# page_url = "https://place.map.kakao.com/710342577"


store_names = df["name"]
page_urls = df["url"]
review_results = []

driver = wb.Chrome() # 브라우저 실행

for index, (store_name, page_url) in enumerate(zip(store_names, page_urls)):

    try:

        driver.get(page_url) # 음식점 상세 화면
        time.sleep(1) # 1초 쉬기


        # total_num = int((bs(driver.page_source,'lxml')).select("span.color_g")[0].text.split(")")[0][1:])
        
        soup = bs(driver.page_source,'lxml')
        total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수


        more_nums = (int(tot_num.text) - 3) // 5 + 1
        print(f"--------------- {index}. {store_name} 후기 더보기 ---------------")
        for more_num in range(more_nums):
            try:
                btnMore = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[7]/div[3]/a')
                btnMore.click()
                time.sleep(1)
            except:
                btnMore = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[8]/div[3]/a')
                btnMore.click()
                time.sleep(1)


        soup = bs(driver.page_source,'lxml')

        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
        rates = contents_div.find_all(name="span", attrs={"class": "ico_star inner_star"}) # 별점값 목록
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
        users = contents_div.find_all(name="span", attrs={"class": "txt_username"})
        reviews_count = contents_div.find_all(name="span", attrs={"class": "txt_desc"})[0::2]
        avg_rates = contents_div.find_all(name="span", attrs={"class": "txt_desc"})[1::2]
        date = contents_div.find_all(name="span", attrs={"class": "time_write"})
        is_model = df.loc[index, "exemplary_diner"]

        print(f"--------------- {index}. {store_name} 리뷰 크롤링 ---------------")

        for idx in range(len(reviews)):

            # rate, review, user, review_count, avg_rate 지정
            user = users[idx].text
            rate = ((int(rates[idx].get("style")[6:-2]) / 100) * 5)    
            avg_rate = float(avg_rates[idx].text)
            review = reviews[idx].find(name="span").text
            review_count = reviews_count[idx].text
                
            # dict 파일 생성
            review_results.append({"store_name" : store_name,
                                "user" : user,                         
                                "rate" : rate,                                                    
                                "avg_rate" : avg_rate,
                                "review" : review,
                                "review_count" : review_count,
                                "date" : date,
                                "is_model" : is_model})
    
    except:
        print(f"카카오맵에 {store_name} 정보가 없음")



reviews_total_df = pd.DataFrame(review_results)

# url 채운 파일 csv로 저장
reviews_total_df.to_csv("data/reviews_total.csv", index=False, encoding="utf-8-sig")